#Boltzmann machines are used to model how the brain stores and retrieves memories

In [1]:
# Packages

import numpy as np

In [2]:
# Creating a class of the Boltzmann machine that we can just call

class BoltzmannMachine:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible # number of visible layers
        self.num_hidden = num_hidden # number of hidden layers
        self.weights = np.random.normal(0, 1, (num_visible, num_hidden)) # randomly initialising the weights from the standard normal distribution w ~ N(0, 1)
        self.bias_visible = np.zeros(num_visible) # initialing the bias for the visible layers as initially a row vector of zeroes with cardinality = number of visible layers
        self.bias_hidden = np.zeros(num_hidden) # initialing the bias for the hidden layers as initially a row vector of zeroes with cardinality = number of hidden layers

    # The nonlinear activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Implmenting Gibbs sampling
    def gibbs_sampling(self, visible_state):
        hidden_prob = self.sigmoid(np.dot(visible_state, self.weights) + self.bias_hidden) # p(h) = 1 / (1 + exp(-(vi * xi + bi)))
        hidden_state = np.random.binomial(1, hidden_prob) # a random number from the binomial distribution
        visible_prob = self.sigmoid(np.dot(hidden_state, self.weights.T) + self.bias_visible)  # p(v) = 1 / (1 + exp(-(hi * xi + bi)))
        visible_state = np.random.binomial(1, visible_prob) # a random number from the binomial distribution
        return hidden_state, visible_state

    # Training the model
    def train(self, data, num_epochs=100, learning_rate=0.1):
        num_examples = data.shape[0]
        for epoch in range(num_epochs):
            for example in range(num_examples):
                visible_state = data[example]
                hidden_state, visible_state_recon = self.gibbs_sampling(visible_state)
                positive_gradient = np.outer(visible_state, hidden_state)
                negative_gradient = np.outer(visible_state_recon, self.sigmoid(np.dot(visible_state_recon, self.weights) + self.bias_hidden))
                self.weights += learning_rate * (positive_gradient - negative_gradient)
                self.bias_visible += learning_rate * (visible_state - visible_state_recon)
                self.bias_hidden += learning_rate * (hidden_state - self.sigmoid(np.dot(visible_state_recon, self.weights) + self.bias_hidden))
    
    # Generting samples
    def generate_samples(self, num_samples):
        samples = np.zeros((num_samples, self.num_visible))
        visible_state = np.random.binomial(1, 0.5, self.num_visible)
        for sample in range(num_samples):
            hidden_state, visible_state = self.gibbs_sampling(visible_state)
            samples[sample] = visible_state
        return samples

#An example of generating samples

In [3]:
data = np.array([[0, 0, 1, 1], [1, 0, 0, 1], [0, 1, 1, 0], [1, 1, 0, 0]])
bm = BoltzmannMachine(num_visible = 4, num_hidden = 2)
bm.train(data)
samples = bm.generate_samples(num_samples=10)
print(samples)

[[1. 1. 1. 0.]
 [1. 0. 0. 1.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
